In [1]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
from IPython.display import display, clear_output
import time

In [2]:
temp = 10000
cooling_rate = 0.001

In [3]:
def distance(city1, city2):
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

In [4]:
def total_distance(route):
    return sum(distance(cities[route[i]], cities[route[i - 1]]) for i in range(num_cities))

In [5]:
def initial_population(num_individuals, num_cities):
    population = []
    base_route = list(range(num_cities))
    for _ in range(num_individuals):
        random.shuffle(base_route)
        population.append(base_route[:])
    return population

In [6]:
def tournament_selection(population, tournament_size):
    competitors = random.sample(population, tournament_size)
    sorted_competitors = sorted(competitors, key=lambda x: total_distance(x))
    winner = sorted_competitors[0]
    return winner

In [7]:
def recombine(parent1, parent2):
    subset_size = int(len(parent1) / 2)
    subset1 = random.sample(parent1, subset_size)
    subset2 = [city for city in parent2 if city not in subset1]
    child_route = subset1 + subset2
    return child_route

In [8]:
def mutate(route):
    swap_index1, swap_index2 = random.sample(range(len(route)), 2)
    route[swap_index1], route[swap_index2] = route[swap_index2], route[swap_index1]
    return route

In [9]:
def survivor_selection(old_population, offspring, retain_length, random_select_rate=0.05):
    sorted_population = sorted(old_population + offspring, key=total_distance)
    new_population = sorted_population[:retain_length]

    for individual in sorted_population[retain_length:]:
        if random.random() < random_select_rate:
            new_population.append(individual)

    while len(new_population) > len(old_population):
        new_population.pop()

    return new_population

In [10]:
def simulated_annealing(route, temp, cooling_rate):
    current_route = route[:]
    best_route = route[:]
    current_population = initial_population(4, num_cities)
    best_distance = total_distance(route)
    distances = [best_distance]

    while temp > 1:
        new_population = []

        for _ in range(len(current_population)):
            parent1 = tournament_selection(current_population, 3)
            parent2 = tournament_selection(current_population, 3)

            if random.random() < 0.5:
                child = recombine(parent1, parent2)
            else:
                child = mutate(parent1)

            current_distance = total_distance(child)

            if current_distance < best_distance:
                best_distance = current_distance
                best_route = child

            new_population.append(child)

        current_population = survivor_selection(current_population, new_population, len(current_population))
        temp *= (1 - cooling_rate)

    return best_route

In [11]:
def start_algorithm():
    global num_cities, cities, route, initial_route

    inputs = []
    for i in range(5):
        try:
            x = float(entries[i].get())
            y = float(entries[i+5].get())
            inputs.append((x, y))
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numerical inputs for all pairs of indices (x, y).")
            return

    num_cities = 5
    cities = {i: inputs[i] for i in range(num_cities)}
    route = list(cities.keys())
    random.shuffle(route)
    initial_route = route[:]

    best_route = simulated_annealing(route, temp, cooling_rate)

    result_label.config(text="Best Route: " + str(best_route) + "\nTotal Distance: " + str(total_distance(best_route)))

In [43]:
root = tk.Tk()
root.title("Simulated Annealing TSP Solver")
root.geometry("460x260")
labels = ["Index 1 (x)", "Index 2 (x)", "Index 3 (x)", "Index 4 (x)", "Index 5 (x)",
        "Index 1 (y)", "Index 2 (y)", "Index 3 (y)", "Index 4 (y)", "Index 5 (y)"]

entries = [tk.Entry(root) for _ in range(10)]
for i in range(5):
        tk.Label(root, text=labels[i]).grid(row=i, column=0, padx=5, pady=5)
        entries[i].grid(row=i, column=1, pady=5)
for i in range(5,10):
        y=i-5
        tk.Label(root, text=labels[i]).grid(row=y, column=2, padx=20, pady=5)
        entries[i].grid(row=y, column=3, pady=5)

start_button = tk.Button(root, text="Start Algorithm", command=start_algorithm)
start_button.grid(row=6,columnspan=4,padx=20,pady=10)

result_label = tk.Label(root, text="")
result_label.grid(row=7,columnspan=4,padx=20,pady=10)

root.mainloop()